In [66]:
import pandas as pd
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import os
import urllib.parse


# 공통 파라미터 설정
base_url = "https://search.naver.com/search.naver"
common_params = {
    'where': 'news',
    'query': '삼성전자',
    'sm': 'tab_opt',
    'sort': '1',
    'photo': '0',
    'field': '0',
    # 'pd': '3',
    'pd': '7',
    # 'ds': '2024.04.23',
    # 'de': '2024.04.23',
    'docid': '',
    'related': '0',
    'mynews': '1',
    'office_type': '0',
    'office_section_code': '0',
    'news_office_checked': '',
    # 'nso': 'so:dd,p:from20240423to20240423',
    'is_sug_officeid': '0',
    'office_category': '0',
    'service_area': '0',
}

encoded_params = urllib.parse.urlencode(common_params)
full_url = f"{base_url}?{encoded_params}"

print("URL:", full_url)


# Initialize a Chrome webdriver
op = webdriver.ChromeOptions()
op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument("--disable-dev-shm-usage")
# op.add_argument("--window-size=1920x1080")

# service.start()
# driver = webdriver.Remote(service.service_url, options=op)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager(version="119.0.6045.105").install()), options=op)
service = Service("./chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=op)
SCROLL_PAUSE_TIME = 0.5
driver.get(full_url)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


# Wait for the dynamic content to load
time.sleep(1)

# Parse the webpage with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

URL: https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=1&photo=0&field=0&pd=7&docid=&related=0&mynews=1&office_type=0&office_section_code=0&news_office_checked=&is_sug_officeid=0&office_category=0&service_area=0


In [68]:
articles = soup.select("#main_pack > section.sc_new.sp_nnews._fe_news_collection._prs_nws > div.api_subject_bx > div.group_news > ul")
print(len(articles[0]))
news_links = []
for i in range(len(articles[0])):
    article = articles[0].select(f"#sp_nws{i+1} > div > div > div.news_info > div.info_group > a:nth-child(4)")
    if article == []:
        article = articles[0].select(f"#sp_nws{i+1} > div > div > div.news_info > div.info_group > a:nth-child(3)")
        if article == []:
            continue
    news_links.append(article[0]["href"])
driver.close()

22


In [69]:
news_links

['https://n.news.naver.com/mnews/article/312/0000659398?sid=106',
 'https://n.news.naver.com/mnews/article/437/0000389757?sid=101',
 'https://n.news.naver.com/mnews/article/014/0005175655?sid=110',
 'https://n.news.naver.com/mnews/article/243/0000058844?sid=101',
 'https://n.news.naver.com/mnews/article/119/0002823220?sid=101']

In [33]:
for link in news_links:
    driver.get(link)

[<li class="bx" id="sp_nws2"> <div class="news_wrap api_ani_send"> <div class="news_area"> <div class="news_info"> <div class="api_save_group _keep_wrap"> <a aria-pressed="false" class="btn_save _keep_trigger" data-url="http://www.goodkyung.com/news/articleView.html?idxno=235151" href="#" onclick="tCR('a=nws*f.kep&amp;r=2&amp;i=88172beb_000000000000000000159574&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=nws*f.kepoff&amp;r=2&amp;i=88172beb_000000000000000000159574&amp;u=javascript" data-cr-on="a=nws*f.kepon&amp;r=2&amp;i=88172beb_000000000000000000159574&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=nws*f.kephome&amp;r=2&amp;i=88172beb_000000000000000000159574&amp;u='+urlencode(this.href));" target="_blank">Keep 바로가기</a> </div></d

In [65]:
driver.quit()